# Resequence Plan
- Calculating number of samples at each processing step and making plan for resequencing

In [415]:
import pandas as pd
import os
import numpy as np

In [416]:
# load in csv file 
reseq=pd.read_csv('resequence_combined.csv')

In [417]:
# view first lines and dimensions 
print(reseq.shape)
reseq.head()

(217, 28)


,Tubelabel_species,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
0,052022_BEL_CBC_T3_50_MCAV,MCAV,2022,Diseased_Margin,56,6_14_2022,0,_,_,_,...,_,extracted dna - pink box,n,enrich,UML_NARWHAL_R1_B3,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B3,DNA_extracted,Only sand left in tissue sample
1,102019_BEL_CBC_T2_35_PSTR,PSTR,2019,Healthy,_,NaN,_,_,_,_,...,_,coral tissue - B2,n,"extract, enrich",UML_NARWHAL_R1_B2,NaN,NaN,UML_NARWHAL_R1_B2,NaN,NaN
2,052022_BEL_CBC_T1_12_MCAV,MCAV,2022,Diseased_Margin,_,6_10_2022,6.2,7_6_2022,0.1,NaN,...,NaN,"DEPLETED, empty enriched tube - orange box, sm...",n,can't - no tissue left,Depleted,DNA_extracted,NaN,Depleted_ UML_NARWHAL_R1_B3,DNA_extracted,Only sand in tissue sample
3,062019_BEL_CBC_T3_7_PAST,PAST,2019,Healthy,_,6_17_2022,13.5,7_6_2022,2.3,NaN,...,NaN,"empty enriched tube - orange box, coral tissue...",n,"extract, enrich",UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN
4,052022_BEL_CBC_T2_13_PSTR,PSTR,2022,Healthy,173,9_12_2023,26.6,10_6_2023,9.16,UML_NARWHAL_R2_B1,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B8,UML_F3,NaN,UML_NARWHAL_R1_B8,UML_NARWHAL_R2_B3,NaN


In [418]:
# set index and remove all duplicates
reseq.set_index('Tubelabel_species', inplace=True)

In [419]:
# check for duplicates 
duplicates=reseq.index.duplicated()
reseq[duplicates]

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
062019_BEL_CBC_T2_21_PAST,PAST,2019,Healthy,_,6_17_2022,1.4,8_8_2022,1.29,NaN,NaN,...,NaN,"enriched & empty tube - orange box, extracted ...",n,"check dna, enrich?",UML_NARWHAL_R1_B1,DNA_extracted,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_1_PAST,PAST,2019,Healthy,_,6_17_2022,2.9,7_6_2022,14.6,NaN,NaN,...,NaN,"2019 enriched, 2022 empty - orange box, 2022 e...",n,"check dna, enrich?",NaN,NaN,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_1_PAST,PAST,2019,Healthy,_,6_17_2022,72,7_14_2022,0.3,NaN,NaN,...,NaN,"2019 enriched, 2022 empty - orange box, 2022 e...",n,"check dna, enrich?",NaN,NaN,NaN,SERC,NaN,NaN
052022_BEL_CBC_T1_10_PSTR,PSTR,2022,Diseased_Margin,141,6_15_2023,11.1,10_17_2023,3.6,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,Ushijima,NaN,NaN
052022_BEL_CBC_T1_11_PSTR,PSTR,2022,Healthy,133,6_15_2023,3.2,10_17_2023,1.5,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,Ushijima,NaN,NaN
062019_BEL_CBC_T1_3_MCAV,MCAV,2019,Healthy,NaN,extracted,NaN,NaN,NaN,NaN,NaN,...,NaN,extracted dna - orange box,n,enrich,UML_NARWHAL_R1_B1,DNA_extracted,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_9_MCAV,MCAV,2019,Healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,UML_NARWHAL_R1_B1,NaN,NaN,SERC,NaN,NaN


In [420]:
reseq['Sample_physical_location_x'].unique()

array(['UML_NARWHAL_R1_B3', 'UML_NARWHAL_R1_B2', 'Depleted',
       'UML_NARWHAL_R1_B1', 'UML_NARWHAL_R1_B8',
       'UML_NARWHAL_R1_B3; Depleted', nan, 'UML_NARWHAL_R1_B5',
       'UML_NARWHAL_R1_B4', 'Depleted '], dtype=object)

In [421]:
reseq['Sample_physical_location_y'].unique()

array(['Depleted_UML_NARWHAL_R1_B3', 'UML_NARWHAL_R1_B2',
       'Depleted_ UML_NARWHAL_R1_B3', 'UML_NARWHAL_R1_B1',
       'UML_NARWHAL_R1_B8', 'Depleted_UML_NARWHAL_R1_B8',
       'UML_NARWHAL_R1_B3', nan, 'Depleted_UML_NARWHAL_R1_B1',
       'UML_NARWHAL_R1_B5', 'SERC', 'UML_NARWHAL_R1_B4', 'Ushijima',
       'Depleted_UML_NARWHAL_R1_B4', 'Depleted_UML_NARWHAL_R1_B5',
       'Depleted__UML_NARWHAL_R1_B2'], dtype=object)

In [422]:
# what are these serc and ushijima samples doing in here?
reseq[reseq['Sample_physical_location_y']=="SERC"]

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
062019_BEL_CBC_T2_21_PAST,PAST,2019,Healthy,_,6_17_2022,1.4,8_8_2022,1.29,NaN,NaN,...,NaN,"enriched & empty tube - orange box, extracted ...",n,"check dna, enrich?",UML_NARWHAL_R1_B1,DNA_extracted,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_1_PAST,PAST,2019,Healthy,_,6_17_2022,2.9,7_6_2022,14.6,NaN,NaN,...,NaN,"2019 enriched, 2022 empty - orange box, 2022 e...",n,"check dna, enrich?",NaN,NaN,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_1_PAST,PAST,2019,Healthy,_,6_17_2022,72,7_14_2022,0.3,NaN,NaN,...,NaN,"2019 enriched, 2022 empty - orange box, 2022 e...",n,"check dna, enrich?",NaN,NaN,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_3_MCAV,MCAV,2019,Healthy,NaN,extracted,NaN,NaN,NaN,NaN,NaN,...,NaN,extracted dna - orange box,n,enrich,UML_NARWHAL_R1_B1,DNA_extracted,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_9_MCAV,MCAV,2019,Healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,UML_NARWHAL_R1_B1,NaN,NaN,SERC,NaN,NaN


In [423]:
reseq.loc[reseq.index=='062019_BEL_CBC_T1_3_MCAV']

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
062019_BEL_CBC_T1_3_MCAV,MCAV,2019,Healthy,NaN,extracted,NaN,NaN,NaN,NaN,NaN,...,NaN,extracted dna - orange box,n,enrich,UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN
062019_BEL_CBC_T1_3_MCAV,MCAV,2019,Healthy,NaN,extracted,NaN,NaN,NaN,NaN,NaN,...,NaN,extracted dna - orange box,n,enrich,UML_NARWHAL_R1_B1,DNA_extracted,NaN,SERC,NaN,NaN


In [424]:
reseq.loc[reseq.index=='062019_BEL_CBC_T1_9_MCAV']

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
062019_BEL_CBC_T1_9_MCAV,MCAV,2019,Healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,UML_NARWHAL_R1_B1,NaN,NaN,UML_NARWHAL_R1_B1,NaN,NaN
062019_BEL_CBC_T1_9_MCAV,MCAV,2019,Healthy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,UML_NARWHAL_R1_B1,NaN,NaN,SERC,NaN,NaN


In [425]:
reseq.loc[reseq.index=='062019_BEL_CBC_T1_1_PAST']

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
062019_BEL_CBC_T1_1_PAST,PAST,2019,Healthy,_,6_17_2022,2.9,7_6_2022,14.6,NaN,NaN,...,NaN,"2019 enriched, 2022 empty - orange box, 2022 e...",n,"check dna, enrich?",NaN,NaN,don't know if its 2019 or 2022,Depleted_UML_NARWHAL_R1_B1,DNA_extracted,Only sand left in tissue sample
062019_BEL_CBC_T1_1_PAST,PAST,2019,Healthy,_,6_17_2022,2.9,7_6_2022,14.6,NaN,NaN,...,NaN,"2019 enriched, 2022 empty - orange box, 2022 e...",n,"check dna, enrich?",NaN,NaN,NaN,SERC,NaN,NaN
062019_BEL_CBC_T1_1_PAST,PAST,2019,Healthy,_,6_17_2022,72,7_14_2022,0.3,NaN,NaN,...,NaN,"2019 enriched, 2022 empty - orange box, 2022 e...",n,"check dna, enrich?",NaN,NaN,NaN,SERC,NaN,NaN


In [426]:
reseq.loc[reseq.index=='062019_BEL_CBC_T2_21_PAST']

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
062019_BEL_CBC_T2_21_PAST,PAST,2019,Healthy,_,6_17_2022,1.4,8_8_2022,1.29,NaN,NaN,...,NaN,"enriched & empty tube - orange box, extracted ...",n,"check dna, enrich?",UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN
062019_BEL_CBC_T2_21_PAST,PAST,2019,Healthy,_,6_17_2022,1.4,8_8_2022,1.29,NaN,NaN,...,NaN,"enriched & empty tube - orange box, extracted ...",n,"check dna, enrich?",UML_NARWHAL_R1_B1,DNA_extracted,NaN,SERC,NaN,NaN


In [427]:
reseq[reseq['Sample_physical_location_y']=="Ushijima"]

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
052022_BEL_CBC_T1_10_PSTR,PSTR,2022,Diseased_Margin,141,6_15_2023,11.1,10_17_2023,3.6,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,Ushijima,NaN,NaN
052022_BEL_CBC_T1_11_PSTR,PSTR,2022,Healthy,133,6_15_2023,3.2,10_17_2023,1.5,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,Ushijima,NaN,NaN


In [428]:
reseq.loc[reseq.index=='052022_BEL_CBC_T1_10_PSTR']

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
052022_BEL_CBC_T1_10_PSTR,PSTR,2022,Diseased_Margin,141,6_15_2023,11.1,10_17_2023,3.6,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN
052022_BEL_CBC_T1_10_PSTR,PSTR,2022,Diseased_Margin,141,6_15_2023,11.1,10_17_2023,3.6,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,Ushijima,NaN,NaN


In [429]:
reseq.loc[reseq.index=='052022_BEL_CBC_T1_11_PSTR']
# not sure how the metagenomics info got mixed in, but the ushijima sample is a healthy probiotics sample from a completely diff coral
# it's correct on the sample spreadsheet so fix here and don't worry about 

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
052022_BEL_CBC_T1_11_PSTR,PSTR,2022,Diseased_Tissue,133,6_15_2023,3.2,10_17_2023,1.5,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN
052022_BEL_CBC_T1_11_PSTR,PSTR,2022,Healthy,133,6_15_2023,3.2,10_17_2023,1.5,UML_NARWHAL_R2_B1,y,...,NaN,NaN,y,NaN,UML_NARWHAL_R1_B3,UML_NARWHAL_R2_B3,NaN,Ushijima,NaN,NaN


In [430]:
# 062019_BEL_CBC_T1_3_MCAV is duplicated here, so remove 'serc' sample
#same for 062019_BEL_CBC_T1_9_MCAV
#same for 062019_BEL_CBC_T1_1_PAST
#same for 062019_BEL_CBC_T2_21_PAST
#same thing for ushijima ones 
conditions = (reseq['Sample_physical_location_y'] == "SERC") | (reseq['Sample_physical_location_y'] == "Ushijima")
conditions.sum()

7

In [431]:
reseq=reseq[~conditions]

In [432]:
reseq.shape

(210, 27)

In [433]:
# view another duplicate
#reseq.loc[reseq.index=='062019_BEL_CBC_T1_1_PAST',"Status_notes"]
# need to just start over on these because idk which one is 2019 and which one is 2022

In [434]:
# check again for duplicates 
duplicates=reseq.index.duplicated()
reseq[duplicates]

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,


In [435]:
#reseq.loc[reseq.index=='052022_BEL_CBC_T2_11_PAST']
# merging entries manually 

In [436]:
# look at unique values in date enriched col 
reseq['Date_Enriched'].unique()

array(['_', '7_6_2022', '10_6_2023', '7_14_2022', '10_17_2023',
       '10_16_2023', '10_11_23', '8_8_2022', '7_1_2022', '6_22_2022', nan,
       '9_27_2023', '9_28_2023', '10_4_2023'], dtype=object)

In [437]:
reseq['ToDo_Resequence'].unique()

array(['enrich', 'extract, enrich', "can't - no tissue left", nan,
       'check dna', 'check dna, enrich?'], dtype=object)

In [438]:
# replace nas with 'extract' since those ones are new samples (haven't been processed at all yet)
reseq['ToDo_Resequence']=reseq['ToDo_Resequence'].fillna('extract')
reseq['ToDo_Resequence'].unique()

array(['enrich', 'extract, enrich', "can't - no tissue left", 'extract',
       'check dna', 'check dna, enrich?'], dtype=object)

In [439]:
# remove rows that contain 'y' (yes) for ready to resequence (don't need to worry about these for now)
filtered_reseq = reseq[reseq['Ready_resequence'] != 'y']
filtered_reseq.head()

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
052022_BEL_CBC_T3_50_MCAV,MCAV,2022,Diseased_Margin,56,6_14_2022,0,_,_,_,_,...,_,extracted dna - pink box,n,enrich,UML_NARWHAL_R1_B3,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B3,DNA_extracted,Only sand left in tissue sample
102019_BEL_CBC_T2_35_PSTR,PSTR,2019,Healthy,_,NaN,_,_,_,_,_,...,_,coral tissue - B2,n,"extract, enrich",UML_NARWHAL_R1_B2,NaN,NaN,UML_NARWHAL_R1_B2,NaN,NaN
052022_BEL_CBC_T1_12_MCAV,MCAV,2022,Diseased_Margin,_,6_10_2022,6.2,7_6_2022,0.1,NaN,NaN,...,NaN,"DEPLETED, empty enriched tube - orange box, sm...",n,can't - no tissue left,Depleted,DNA_extracted,NaN,Depleted_ UML_NARWHAL_R1_B3,DNA_extracted,Only sand in tissue sample
062019_BEL_CBC_T3_7_PAST,PAST,2019,Healthy,_,6_17_2022,13.5,7_6_2022,2.3,NaN,NaN,...,NaN,"empty enriched tube - orange box, coral tissue...",n,"extract, enrich",UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN
052022_BEL_CBC_T3_8_PAST,PAST,2022,Healthy,_,6_17_2022,15.4,7_14_2022,2.2,NaN,NaN,...,NaN,enriched - orange box,n,check dna,UML_NARWHAL_R1_B3; Depleted,?,"depleted, can't find extracted dna or sequence",Depleted_UML_NARWHAL_R1_B3,?,Only sand left in tissue sample


In [440]:
# first view all columns
filtered_reseq.columns

Index(['Species', 'Year', 'Health_status', 'Starting_Weight', 'Date_Extracted',
       'Raw_ng_ul', 'Date_Enriched', 'Microbe_ng_ul', 'Microbe_Location',
       'Microbe_clean_date/n', 'Host_ng_ul', 'Host_Location', 'Host_clean_y/n',
       'Date_Libprep', 'Status ', 'Notes ', 'Seq_date', 'Host_Seq_date',
       'Status_notes', 'Ready_resequence', 'ToDo_Resequence',
       'Sample_physical_location_x', 'Extraction_physical_location_x',
       'Notes_x', 'Sample_physical_location_y',
       'Extraction_physical_location_y', 'Notes_y'],
      dtype='object')

In [441]:
# view notes on what to do for each sample
filtered_reseq['ToDo_Resequence']

Tubelabel_species
052022_BEL_CBC_T3_50_MCAV                    enrich
102019_BEL_CBC_T2_35_PSTR           extract, enrich
052022_BEL_CBC_T1_12_MCAV    can't - no tissue left
062019_BEL_CBC_T3_7_PAST            extract, enrich
052022_BEL_CBC_T3_8_PAST                  check dna
                                      ...          
062019_BEL_CBC_T3_24_PAST                   extract
062019_BEL_CBC_T3_26_PAST                   extract
062019_BEL_CBC_T3_11_MCAV                   extract
052022_BEL_CBC_T1_1_PAST                    extract
052022_BEL_CBC_T1_63_OFAV                   extract
Name: ToDo_Resequence, Length: 149, dtype: object

In [442]:
# look at all options in todo
filtered_reseq['ToDo_Resequence'].unique()

array(['enrich', 'extract, enrich', "can't - no tissue left", 'check dna',
       'check dna, enrich?', 'extract'], dtype=object)

In [443]:
# change 'extract, enrich' to just extract (since all have to be enriched after extraction - just want to make it clearer and more consistent)
filtered_reseq.loc[filtered_reseq['ToDo_Resequence'] == 'extract, enrich', 'ToDo_Resequence'] = 'extract'

In [444]:
# confirm it worked
filtered_reseq['ToDo_Resequence'].unique()

array(['enrich', 'extract', "can't - no tissue left", 'check dna',
       'check dna, enrich?'], dtype=object)

In [445]:
# investigate ones that say check dna
filtered_reseq[filtered_reseq['ToDo_Resequence'].str.contains('check dna', na=False)]


,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
052022_BEL_CBC_T3_8_PAST,PAST,2022,Healthy,_,6_17_2022,15.4,7_14_2022,2.2,NaN,NaN,...,NaN,enriched - orange box,n,check dna,UML_NARWHAL_R1_B3; Depleted,?,"depleted, can't find extracted dna or sequence",Depleted_UML_NARWHAL_R1_B3,?,Only sand left in tissue sample
052022_BEL_CBC_T1_52_PAST,PAST,2022,Diseased_Tissue,_,6_17_2022,8.1,7_14_2022,0.6,NaN,NaN,...,NaN,enriched - orange box,n,check dna,NaN,NaN,NaN,NaN,NaN,was labelled as 2019 in metagenomics tracker?
062019_BEL_CBC_T3_8_MCAV,MCAV,2019,Healthy,_,5_18_2022,7.8,8_8_2022,-16.13,NaN,NaN,...,NaN,"(2x) T3-8 enriched tubes in orange box, 1 empt...",n,"check dna, enrich?",UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN
062019_BEL_CBC_T1_24_MCAV,MCAV,2019,Healthy,177,6_13_2022,7.1,7_1_2022,1.4,NaN,NaN,...,NaN,"enriched - orange box (small amnt left), extra...",n,check dna,UML_NARWHAL_R1_B1,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B1,DNA_extracted,Only sand left in tissue sample
062019_BEL_CBC_T2_16_MCAV,MCAV,2019,Healthy,_,5_18_2022,6.1,8_8_2022,1.89,NaN,NaN,...,NaN,enriched - orange box,n,check dna,Depleted,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B1,DNA_extracted,Only sand left in tissue sample
052022_BEL_CBC_T1_57_MCAV,MCAV,2022,Healthy,_,6_15_2022,75.5,6_22_2022,1.43,NaN,NaN,...,NaN,enriched - orange box,n,check dna,UML_NARWHAL_R1_B3,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B3,DNA_extracted,Only sand left in tissue sample
052022_BEL_CBC_T3_13_MCAV,MCAV,2022,Healthy,_,6_15_2022,13.2,6_22_2022,1,NaN,NaN,...,NaN,enriched - orange box,n,check dna,UML_NARWHAL_R1_B3,DNA_extracted,NaN,UML_NARWHAL_R1_B3,DNA_extracted,NaN
052022_BEL_CBC_T3_19_MCAV,MCAV,2022,Healthy,_,6_15_2022,38.2,6_22_2022,6.3,NaN,NaN,...,NaN,enriched - orange box,n,check dna,UML_NARWHAL_R1_B3,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B3,DNA_extracted,Only sand left in tissue sample
052022_BEL_CBC_T3_60_MCAV,MCAV,2022,Diseased_Tissue,187,6_14_2022,4.4,8_8_2022,2.72,NaN,NaN,...,NaN,enriched - orange box,n,check dna,UML_NARWHAL_R1_B3,DNA_extracted,NaN,UML_NARWHAL_R1_B3,DNA_extracted,NaN


In [446]:
# they have all been enriched, so changing to check enriched dna 
filtered_reseq.loc[filtered_reseq['ToDo_Resequence'].str.contains('check dna', case=False, na=False), 'ToDo_Resequence'] = 'check enriched dna'
# branae can check dna of these 

In [447]:
# investigate depleted statuses 
condition=filtered_reseq['ToDo_Resequence']=="can't - no tissue left"
depleted=filtered_reseq.loc[condition,["Status_notes","ToDo_Resequence","Sample_physical_location_y","Notes_y"]]
depleted

,Status_notes,ToDo_Resequence,Sample_physical_location_y,Notes_y
Tubelabel_species,,,,
052022_BEL_CBC_T1_12_MCAV,"DEPLETED, empty enriched tube - orange box, sm...",can't - no tissue left,Depleted_ UML_NARWHAL_R1_B3,Only sand in tissue sample
052022_BEL_CBC_T2_10_MCAV,"DEPLETED, empty enriched tube - orange box, em...",can't - no tissue left,Depleted_UML_NARWHAL_R1_B8,Only sand left in tissue sample
052022_BEL_CBC_T1_40_MCAV,DEPLETED,can't - no tissue left,Depleted_UML_NARWHAL_R1_B3,Only sand left in tissue sample
052022_BEL_CBC_T1_70_MCAV,"DEPLETED, empty enriched tube - orange box",can't - no tissue left,Depleted_UML_NARWHAL_R1_B3,Only sand left in tissue sample
052022_BEL_CBC_T1_13_MCAV,"DEPLETED, empty enriched tube - orange box, em...",can't - no tissue left,Depleted_UML_NARWHAL_R1_B3,Only sand in tissue sample


In [448]:
#view status notes more closely
for row in range(len(depleted)):
    print(depleted.iloc[row, 0])

DEPLETED, empty enriched tube - orange box, small amnt extracted dna - pink box 
DEPLETED, empty enriched tube - orange box, empty extracted - pink box 
DEPLETED
DEPLETED, empty enriched tube - orange box 
DEPLETED, empty enriched tube - orange box, empty extracted - pink box,  


In [449]:
# going to have to filter tubes and try to re-extract 
# replace all to say extract_from_depleted in toDO 
depleted.iloc[1:5,1]="extract_from_depleted"

In [450]:
# except first row, can try to check enriched DNA 
depleted.iloc[0,1]="check enriched dna"

In [451]:
depleted['ToDo_Resequence'].unique()

array(['check enriched dna', 'extract_from_depleted'], dtype=object)

In [452]:
# reset so indices are sample names 
#filtered_reseq.set_index('Tubelabel_species', inplace=True)
#depleted.set_index('Tubelabel_species', inplace=True)  

# should be set already

In [453]:
filtered_reseq.loc[depleted.index.intersection(filtered_reseq.index), 'ToDo_Resequence'] = \
    depleted.loc[depleted.index.intersection(filtered_reseq.index), 'ToDo_Resequence']

In [454]:
filtered_reseq['ToDo_Resequence'].unique()

array(['enrich', 'extract', 'check enriched dna', 'extract_from_depleted'],
      dtype=object)

In [455]:
# original values
# array(['enrich', 'extract, enrich', "can't - no tissue left", 'check dna',
#       'check dna, enrich?', 'extract'], dtype=object)

In [456]:
# update original with filtered  
reseq.update(filtered_reseq) 

In [457]:
#view 
reseq['ToDo_Resequence'].unique()

array(['enrich', 'extract', 'check enriched dna', 'extract_from_depleted'],
      dtype=object)

In [478]:
# total number of samples to process
print('total number of samples to process',reseq.shape[0])

total number of samples to process 210


In [459]:
# count sample totals for each category 
todo_counts = reseq['ToDo_Resequence'].value_counts()

# Print the counts for each unique entry
print("# of samples for each step in 'ToDo_Resequence':")
print(todo_counts)

# of samples for each step in 'ToDo_Resequence':
ToDo_Resequence
extract                  171
check enriched dna        21
enrich                    14
extract_from_depleted      4
Name: count, dtype: int64


In [479]:
# if i do 8 extractions a day how many days will this take
print("# of days to finish extractions",171/8)
# 22 days
print("# of weeks to finish extractions",22/7)
print("including depleted extractions", 175/8,"days")
#....try to do more each day 

# of days to finish extractions 21.375
# of weeks to finish extractions 3.142857142857143
including depleted extractions 21.875 days


In [481]:
print("# of days to finish enrichments",(14+175)/8)
print("# of weeks to prep all",(22+24)/7)
# i definitely didnt include all the samples oops 

# of days to finish enrichments 23.625
# of weeks to prep all 6.571428571428571


In [469]:
# how many to re-extract
re_extract = (reseq['ToDo_Resequence'] == "extract") & (reseq['Date_Extracted'].notna()) & (~reseq['Ready_resequence'].str.contains('y', na=False))
reseq[re_extract]

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
062019_BEL_CBC_T3_7_PAST,PAST,2019,Healthy,_,6_17_2022,13.5,7_6_2022,2.3,NaN,NaN,...,NaN,"empty enriched tube - orange box, coral tissue...",n,extract,UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN
052022_BEL_CBC_T2_11_PAST,PAST,2022,Healthy,_,6_17_2022,141.7,7_14_2022,1.8,NaN,NaN,...,NaN,very little coral tissue - B1,n,extract,UML_NARWHAL_R1_B8,DNA_extracted,labeleld as 2019 in metagenomics tracker?,Depleted_UML_NARWHAL_R1_B8,DNA_extracted,Only sand left in tissue sample; labelled as 2...
122022_BEL_CBC_T4_13_PSTR,PSTR,2022,Diseased_Tissue,178.7,7/3/24,6.21,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,extract,UML_NARWHAL_R1_B5,NaN,NaN,UML_NARWHAL_R1_B5,NaN,NaN
122022_BEL_CBC_T4_36_PSTR,PSTR,2022,Diseased_Tissue,177.3,7/3/24,34.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,extract,UML_NARWHAL_R1_B5,NaN,NaN,UML_NARWHAL_R1_B5,NaN,NaN
122022_BEL_CBC_T1_132_PSTR,PSTR,2022,Diseased_Tissue,171.6,7/3/24,29.4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,extract,UML_NARWHAL_R1_B4,NaN,NaN,UML_NARWHAL_R1_B4,NaN,NaN
122022_BEL_CBC_T2_117_PSTR,PSTR,2022,Diseased_Tissue,280.8,7/3/24,22.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,extract,UML_NARWHAL_R1_B4,NaN,NaN,UML_NARWHAL_R1_B4,NaN,NaN


In [470]:
# how many to re-extract
re_enrich = (reseq['ToDo_Resequence'] == "enrich") & (reseq['Date_Extracted'].notna()) & (~reseq['Ready_resequence'].str.contains('y', na=False))
reseq[re_enrich]
# are we not confident about any of these? should we re-extract any?

,Species,Year,Health_status,Starting_Weight,Date_Extracted,Raw_ng_ul,Date_Enriched,Microbe_ng_ul,Microbe_Location,Microbe_clean_date/n,...,Host_Seq_date,Status_notes,Ready_resequence,ToDo_Resequence,Sample_physical_location_x,Extraction_physical_location_x,Notes_x,Sample_physical_location_y,Extraction_physical_location_y,Notes_y
Tubelabel_species,,,,,,,,,,,,,,,,,,,,,
052022_BEL_CBC_T3_50_MCAV,MCAV,2022,Diseased_Margin,56,6_14_2022,0,_,_,_,_,...,_,extracted dna - pink box,n,enrich,UML_NARWHAL_R1_B3,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B3,DNA_extracted,Only sand left in tissue sample
062019_BEL_CBC_T3_25_PAST,PAST,2019,Healthy,_,6_17_2022,14.9,7_14_2022,21.2,NaN,NaN,...,NaN,extracted dna - pink box,n,enrich,UML_NARWHAL_R1_B1,NaN,NaN,UML_NARWHAL_R1_B1,NaN,NaN
052022_BEL_CBC_T3_48_MCAV,MCAV,2022,Diseased_Margin,155,6_14_2022,38.3,7_6_2022,6.4,NaN,NaN,...,NaN,empty enriched tube & no date enriched tube - ...,n,enrich,UML_NARWHAL_R1_B3,DNA_extracted,NaN,UML_NARWHAL_R1_B3,DNA_extracted,NaN
052022_BEL_CBC_T3_49_MCAV,MCAV,2022,Diseased_Tissue,142,6_14_2022,41.9,7_6_2022,5.8,NaN,NaN,...,NaN,extracted dna - pink box,n,enrich,UML_NARWHAL_R1_B3,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B3,DNA_extracted,Only sand left in tissue sample
052022_BEL_CBC_T3_51_MCAV,MCAV,2022,Diseased_Tissue,104,6_14_2022,30.8,7_6_2022,1.5,NaN,NaN,...,NaN,extracted dna - pink box,n,enrich,UML_NARWHAL_R1_B3,DNA_extracted,NaN,Depleted_UML_NARWHAL_R1_B3,DNA_extracted,Only sand left in tissue sample
062019_BEL_CBC_T3_9_MCAV,MCAV,2019,Healthy,162,6_13_2022,0.8,7_1_2022,1.3,NaN,NaN,...,NaN,"empty enriched tube - orange box, empty extrac...",n,enrich,UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN
052022_BEL_CBC_T3_40_MCAV,MCAV,2022,Diseased_Margin,139,6_14_2022,24.7,7_6_2022,7.5,NaN,NaN,...,NaN,"empty enriched tube - orange box, extracted dn...",n,enrich,UML_NARWHAL_R1_B3,DNA_extracted,NaN,UML_NARWHAL_R1_B3,DNA_extracted,NaN
052022_BEL_CBC_T2_69_OANN,OANN,2022,Healthy,134,10_3_2023,133.1,NaN,NaN,NaN,NaN,...,NaN,extracted dna - B3,n,enrich,UML_NARWHAL_R1_B8,UML_NARWHAL_R2_B3,NaN,Depleted_UML_NARWHAL_R1_B8,UML_NARWHAL_R2_B3,Only sand left in tissue sample
062019_BEL_CBC_T1_3_MCAV,MCAV,2019,Healthy,NaN,extracted,NaN,NaN,NaN,NaN,NaN,...,NaN,extracted dna - orange box,n,enrich,UML_NARWHAL_R1_B1,DNA_extracted,NaN,UML_NARWHAL_R1_B1,DNA_extracted,NaN


In [483]:
# export updated reseq 
reseq.to_csv('resequence_combined.csv', index=True) 